In [1]:
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime

### Loading Bill Line dust climatology: 

In [2]:
df = pd.read_csv('Dust_LatLon_Drobo_Complete_June2022_Jesse - Data LatLon.csv')

In [3]:
df_dust = df[df["Jesse Check"] == "x"]

In [4]:
df_dust

,Date (YYYYMMDD),YYYY,M,D,latitude,longitude,start time (UTC),Jesse Check,Notes,New GOES Notes,Julian day,GOES Raw,GOES Images,Unnamed: 13,Unnamed: 14,Unnamed: 15
24,20010116.0,2001.0,1.0,16.0,30.3,-107.4,1900.0,x,NaN,Northern CHH,NaN,NaN,NaN,NaN,NaN,20010504.0
25,20010116.0,2001.0,1.0,16.0,29.3,-106.9,1900.0,x,NaN,Central CHH,NaN,NaN,NaN,NaN,NaN,20010505.0
26,20010116.0,2001.0,1.0,16.0,28.6,-106.7,1930.0,x,NaN,"Subtle, Central CHH",NaN,NaN,NaN,NaN,NaN,20010516.0
27,20010125.0,2001.0,1.0,25.0,31.1,-107.9,1930.0,x,NaN,Northern CHH,NaN,NaN,NaN,NaN,NaN,20010517.0
28,20010127.0,2001.0,1.0,27.0,31.2,-107.9,2000.0,x,NaN,"Cloud coverage, northern CHH",NaN,NaN,NaN,NaN,NaN,20010520.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2349,20201223.0,2020.0,12.0,23.0,38.0,-101.7,1530.0,x,NaN,KS,NaN,NaN,NaN,NaN,NaN,NaN
2350,20201229.0,2020.0,12.0,29.0,29.3,-106.8,1930.0,x,NaN,CHH,NaN,NaN,NaN,NaN,NaN,NaN
2351,20201229.0,2020.0,12.0,29.0,28.9,-106.9,1930.0,x,NaN,CHH,NaN,NaN,NaN,NaN,NaN,NaN
2352,20201229.0,2020.0,12.0,29.0,29.0,-104.8,2000.0,x,NaN,CHH,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_dust_CHH = df_dust[(df_dust["latitude"] > 30) & (df_dust["latitude"] < 33) & (df_dust["longitude"] < -105) & (df_dust["longitude"] > -110)]

In [6]:
dates = df_dust_CHH['Date (YYYYMMDD)'].unique()

In [8]:
dates = np.array(dates.astype(int).astype(str))

Save the dust dates as a CSV:

In [10]:
#--- Needs to be <600 for NARR composite app
len(dates)

378

In [11]:
np.savetxt('dust_dates.txt', dates, fmt='%s')

### Using monthly-mean geopotential height: 
* climatology averaged between 1981 and 2010
* https://psl.noaa.gov/data/gridded/data.narr.html

In [ ]:
file = 'hgt_mon_ltm.nc'

In [ ]:
hgt_ds = xr.open_dataset(file)

In [ ]:
hgt_ds

In [ ]:
level = 500  #hPa
time = hgt_ds.time[0] #month

hgt_ds_sel = hgt_ds.sel(level=level, time=time)

In [ ]:
hgt_ds_sel

In [ ]:
projection=ccrs.PlateCarree()
fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})
cmap = plt.cm.Spectral_r
levels = np.linspace(5000, 6000, 31)

ax.set_extent([-125, -97, 24, 49], crs=ccrs.PlateCarree())
c=ax.contourf(hgt_ds_sel.lon, hgt_ds_sel.lat, hgt_ds_sel.hgt, cmap=cmap, levels=levels, extend='both')
clb=plt.colorbar(c, shrink=0.3, pad=0.02, ax=ax)
ax.set_title('Monthly-mean geopotential height (Jan)')
clb.set_label('meters (m)')


ax.add_feature(cfeature.STATES)

### Using daily-mean geopotential height: 
* climatology averaged between 1981 and 2010
* https://psl.noaa.gov/data/gridded/data.narr.html

In [ ]:
file = 'hgt_day_ltm.nc'
hgt_ds = xr.open_dataset(file)
hgt_ds

In [ ]:
level = 500  #hPa
time = hgt_ds.time[0] #julian day

hgt_ds_sel = hgt_ds.sel(level=level, time=time)

In [ ]:
hgt_ds_sel

Filter the NARR data to only dust days:
* maybe I should be using the hgt from each of these specific datetimes, instead of the average

In [ ]:
df_dust["Date (YYYYMMDD)"]

In [ ]:
specific_days = ['2023-01-01', '2023-02-15', '2023-04-30']

specific_dates = pd.to_datetime(specific_days)

filtered_ds = ds.sel(time=specific_dates)

In [ ]:
projection=ccrs.PlateCarree()
fig,ax=plt.subplots(1, figsize=(12,12),subplot_kw={'projection': projection})
cmap = plt.cm.Spectral_r
levels = np.linspace(5000, 6000, 31)

ax.set_extent([-125, -97, 24, 49], crs=ccrs.PlateCarree())
c=ax.contourf(hgt_ds_sel.lon, hgt_ds_sel.lat, hgt_ds_sel.hgt, cmap=cmap, levels=levels, extend='both')
clb=plt.colorbar(c, shrink=0.3, pad=0.02, ax=ax)
ax.set_title('Daily-mean geopotential height')
clb.set_label('meters (m)')


ax.add_feature(cfeature.STATES)